# Coleta de dados
Nesta seção iremos coletar os dados por meio de web scraping e apis de outros sites

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
prefix = 'https://www.transfermarkt.com'
def getSoup(url):
    res = requests.get(url, headers = headers)
    return BeautifulSoup(res.content, 'html.parser')


### Função para realizar o web scraping das tableas de transferências no transfermarkt

In [30]:
def getClubUrl(url):
    url = url.replace('/transfers', '')  # Remove a substring '/transfers'
    match = re.search(r'/verein/(\d+)', url)
    if(not match):
        return "invalid";
    verein_id = match.group(1)  # Obtém o número após "/verein/"
    new_url = match.string[:match.start()] + \
        '/startseite' '/verein/' + verein_id
    return new_url


def eurValue(string):
    # Expressão regular para verificar se começa com '€' e termina com 'm' ou 'k'
    padrao = r'^€.*[mk]$'
    if re.match(padrao, string):
        return True
    else:
        return False


def eurToInt(string):
    valor_str = re.sub(r'[€mk]', '', string)  # Remove '€', 'm' e 'k' da string
    # Substitui a vírgula pelo ponto como separador decimal e converte para float
    valor_float = float(valor_str.replace(',', '.'))
    if 'm' in string:
        valor_int = int(valor_float * 1000000)  # Converte para milhões
    elif 'k' in string:
        valor_int = int(valor_float * 1000)  # Converte para milhares
    else:
        valor_int = int(valor_float)
    return valor_int


def getTransfersFromUrl(url, year):
    transferTable = []
    page = getSoup(url.format(year))
    tables = page.find_all('div', {'class': 'box'})
    tables = tables[3:]
    for table in tables:
        currTeamUrl = getClubUrl(table.find('a').get('href'))
        if(currTeamUrl == "invalid"):
            break;
        currTeam = table.find('a').get('title')
        arrivalsAndDepartures = table.find_all('table')

        arrivals = arrivalsAndDepartures[0].find_all('tr', recursive=True)
        arrivals = arrivals[1:]
        for transfer in arrivals:
            transfer = transfer.find_all('td')
            player = transfer[0].find('a')
            if isinstance(player, type(None)):
                continue
            playerName = player.text
            playerLink = player.get('href')
            playerAge = transfer[1].text
            playerPos = transfer[4].text
            if(transfer[2].find('img')):
                playerCountry = transfer[2].find('img').get('alt')
            else:
                playerCountry = "none"
            transferFee = transfer[8].text
            if (not eurValue(transferFee)):
                continue
            if (not eurValue(transfer[5].text)):
                playerMarketValue = 0
            else:
                playerMarketValue = eurToInt(transfer[5].text)
            transferFee = eurToInt(transferFee)
            if (transferFee < 10):
                continue
            clubLeftUrl = getClubUrl(transfer[6].find('a').get('href'))
            clubJoinedUrl = currTeamUrl
            clubLeft = transfer[6].find('a').get('title')
            clubJoined = currTeam
            teamTransferType = "in"

            transferTable.append([playerName, playerLink, playerAge, playerPos, playerCountry,
                                  playerMarketValue, transferFee, clubLeft, clubLeftUrl, clubJoined, clubJoinedUrl, teamTransferType])

        departures = arrivalsAndDepartures[1].find_all(
            'tr', recursive=True)
        departures = departures[1:]
        for transfer in departures:
            transfer = transfer.find_all('td')
            player = transfer[0].find('a')
            if isinstance(player, type(None)):
                continue
            playerName = player.text
            playerLink = player.get('href')
            playerAge = transfer[1].text
            playerPos = transfer[4].text

            if(transfer[2].find('img')):
                playerCountry = transfer[2].find('img').get('alt')
            else:
                playerCountry = "none"
            transferFee = transfer[8].text
            if (not eurValue(transferFee)):
                continue
            if (not eurValue(transfer[5].text)):
                playerMarketValue = 0
            else:
                playerMarketValue = eurToInt(transfer[5].text)
            transferFee = eurToInt(transferFee)
            if (transferFee < 10):
                continue
            clubJoinedUrl = getClubUrl(transfer[6].find('a').get('href'))
            clubLeftUrl = currTeamUrl
            clubLeft = currTeam
            clubJoined = transfer[6].find('a').get('title')
            teamTransferType = "out"

            transferTable.append([playerName, playerLink, playerAge, playerPos, playerCountry,
                                  playerMarketValue, transferFee, clubLeft, clubLeftUrl, clubJoined, clubJoinedUrl, teamTransferType])

    columns = ['playerName', 'playerLink', 'playerAge', 'playerPos', 'playerCountry',
               'playerMarketValue', 'transferFee', 'clubLeft', 'clubLeftUrl', 'clubJoined', 'clubJoinedUrl', 'teamTransferType']
    final_df = pd.DataFrame(transferTable, columns=columns)
    final_df['season'] = year
    final_df['playerAge'] = final_df['playerAge'].apply(
        lambda x: re.sub(r'\D+', '', str(x)))
    final_df['playerAge'] = final_df['playerAge'].astype(int)
    final_df['valuation'] = final_df['playerMarketValue'] - final_df['transferFee']
    return final_df

### Função para fazer o web scraping das tabelas de transferências de um determinado período e concatenar elas

In [31]:
def getTransfersInRange(begin, end, url):
    curr_df = getTransfersFromUrl(url, begin)
    for i in range(begin+1, end+1):
        curr_df = pd.concat(
            [curr_df, getTransfersFromUrl(url, i)]).reset_index(drop=True)
    return curr_df

### Obtendo dados das transferências para um range de temporadas da Premier League (demora 3 a 6 segundos por temporada, depende da internet)

In [26]:
pl = 'https://www.transfermarkt.com/premier-league/transfers/wettbewerb/GB1/plus/?saison_id={}&s_w=&leihe=0&intern=0'
ll = 'https://www.transfermarkt.com/laliga/transfers/wettbewerb/ES1/plus/?saison_id={}&s_w=&leihe=0&intern=0'
bl = 'https://www.transfermarkt.com/bundesliga/transfers/wettbewerb/L1/plus/?saison_id={}&s_w=&leihe=0&intern=0'
sa = 'https://www.transfermarkt.com/serie-a/transfers/wettbewerb/IT1/plus/?saison_id={}&s_w=&leihe=0&intern=0'
l1 = 'https://www.transfermarkt.com/ligue-1/transfers/wettbewerb/FR1/plus/?saison_id={}&s_w=&leihe=0&intern=0'

In [27]:
pl_transfers = getTransfersInRange(2000, 2023, pl)
ll_transfers = getTransfersInRange(2000, 2023, ll)
bl_transfers = getTransfersInRange(2000, 2023, bl)
sa_transfers = getTransfersInRange(2000, 2023, sa)
l1_transfers = getTransfersInRange(2000, 2023, l1)

In [28]:
pl_transfers = pl_transfers.drop_duplicates(subset=['playerLink', 'playerAge', 'transferFee'])
pl_transfers.to_csv('data/transfers/pl_transfers.csv', index=False)

ll_transfers = ll_transfers.drop_duplicates(subset=['playerLink', 'playerAge', 'transferFee'])
ll_transfers.to_csv('data/transfers/ll_transfers.csv', index=False)

bl_transfers = bl_transfers.drop_duplicates(subset=['playerLink', 'playerAge', 'transferFee'])
bl_transfers.to_csv('data/transfers/bl_transfers.csv', index=False)

sa_transfers = sa_transfers.drop_duplicates(subset=['playerLink', 'playerAge', 'transferFee'])
sa_transfers.to_csv('data/transfers/sa_transfers.csv', index=False)

l1_transfers = l1_transfers.drop_duplicates(subset=['playerLink', 'playerAge', 'transferFee'])
l1_transfers.to_csv('data/transfers/l1_transfers.csv', index=False)